In [1]:
# importing
import os
from PyPDF2 import PdfReader
from scidownl import scihub_download


In [2]:
# folder path
PROJECT_DIR = os.path.dirname(os.getcwd())
DATA_DIR = os.path.join(PROJECT_DIR, 'data')
PAPER_DIR = os.path.join(DATA_DIR, 'paper')

# file path
DOI_FILE = os.path.join(PAPER_DIR, 'doi_list.txt')
DOI_FAIL = os.path.join(PAPER_DIR, 'doi_fail.txt')

In [3]:
# Initialize empty list to track doi
doi_list = []

# read text file that consist all doi
with open(DOI_FILE, 'r') as f:
    for line in f:
        doi_list.append(line.rstrip())

In [4]:
def checkFile(fullfile):
    with open(fullfile, 'rb') as f:
        try:
            pdf = PdfReader(f)
            info = pdf.metadata
            if info:
                return True
            else:
                return False
        except:
            return False

In [ ]:
# Define paper type
paper_type = "doi"

# Optional proxy configuration for download requests
proxies = {
    'http': 'socks5://127.0.0.1:7890'
}

# Initialize an empty list to track failed downloads
track_fail = []

# Iterate through each DOI in the provided list
for doi in doi_list:
    # Extract filename from the last element of the DOI URL path
    name_file = doi.split("/")[-1]
    # Append ".pdf" extension to create the output filename
    name_file = name_file + ".pdf"

    # Construct the full output path for the downloaded paper
    path_ = os.path.join(PAPER_DIR, name_file)

    # Check if the paper has already been downloaded
    if not os.path.exists(path_):
        # Attempt to download the paper using scihub_download function
        scihub_download(doi, paper_type=paper_type, out=path_, proxies=proxies)

    # Track download failures
    if not os.path.exists(path_):
        # Append the failed DOI to the track_fail list
        track_fail.append(doi)

    # Check if file is corrupted
    if os.path.exists(path_):
        if not checkFile(path_):
            track_fail.append(doi)
            os.remove(path_)
    # Check if file is corrupted


# Open the DOI failure log file in append mode for logging failed downloads
with open(DOI_FAIL, 'a') as f:
    # Write each failed DOI to the log file
    for fail in track_fail:
        f.write(fail)
        # Add a newline character after each DOI entry
        f.write("\n")

: 